In [1]:
import os

In [2]:
import os
import json
import numpy as np
from rank_bm25 import BM25Okapi

In [3]:
import spacy
nlp = spacy.load("en_core_sci_scibert")

In [4]:
pwd

'/Users/shrut/Work/NewData/IR'

In [5]:
results = {}
def get_relevant_sentences(cs, num_tfidf=10):
    global results

    for pmcid in cs:
        if pmcid.startswith('0'):
            curr_ref_file = pmcid.split('_')[1]
        CURR_PMC = pmcid

        with open(f'Sentences/{curr_ref_file}.json') as f:
            sentences = json.load(f)

        original_sentences = sentences
        sentences = [[str(w) for w in s.split()] for s in sentences]
        bm25 = BM25Okapi(sentences)


        results[CURR_PMC] = {}
        idx = 0
        for file, d in cs[pmcid].items():
            if len(d['support']) == 0:
                continue
            claim = con[pmcid][file]
            idx += 1
            
            y = [str(w) for w in claim.split()]
            scores = bm25.get_scores(y)
            sorted_scores = np.argsort(scores)[::-1]
            results[CURR_PMC][file] = [original_sentences[i] for i in sorted_scores[:10]]


In [4]:
def calculate_recall(cs, results):
    def overlap(start, end, s, e):
        return start <= e and s <= end

    for pmcid in results:
        if pmcid.startswith('0'):
            curr_ref_file = pmcid.split('_')[1]
        with open(f'Articles/{curr_ref_file}.txt') as f:
            content = f.read()
        for file in results[pmcid]:
            indices = [[content.find(s.strip()), content.find(s.strip())+len(s.strip())] for s in results[pmcid][file]]
            count = 0
            support_indices = [[content.find(s.strip()), content.find(s.strip())+len(s.strip())] for s in cs[pmcid][file]['support']]
            for start, end in support_indices:
                for s, e in indices:
                    if overlap(start, end, s, e):
                        count += 1
            cs[pmcid][file]['count'] = count
    res = 0
    total = 0
    for pmcid in cs:
        for file in cs[pmcid]:
            if 'count' in cs[pmcid][file]:
                res += (cs[pmcid][file]['count'] / len(cs[pmcid][file]['support']))
                total += 1

    return res/total

In [6]:
with open('recon_cs.json') as f:
    cs = json.load(f)

In [7]:
with open('recon_cs_no_title.json') as f:
    cst = json.load(f)

In [7]:
with open('Eugene_ContextAndSupport.json') as f:
    cs1 = json.load(f)

In [8]:
with open('Eugene_ContextAndSupport_No_Title.json') as f:
    cs2 = json.load(f)

In [33]:
with open('Degaulle_ContextAndSupport.json') as f:
    cs3 = json.load(f)

In [34]:
with open('Degaulle_ContextAndSupport_No_Title.json') as f:
    cs4 = json.load(f)

In [30]:
results = {}
get_relevant_sentences(cs1)
calculate_recall(cs1, results)

0.23517665130568363

In [31]:
results = {}
get_relevant_sentences(cs2)
calculate_recall(cs2, results)

0.3897342995169083

In [39]:
results = {}
get_relevant_sentences(cs3)
calculate_recall(cs3, results)

0.38192090395480227

In [40]:
results = {}
get_relevant_sentences(cs4)
calculate_recall(cs4, results)

0.4023809523809524

In [10]:
results = {}
get_relevant_sentences(cs)
calculate_recall(cs, results)

0.35577610872675247

In [11]:
results = {}
get_relevant_sentences(cst)
calculate_recall(cst, results)

0.4491051454138701

In [10]:
with open("sent_context.json") as f:
    con = json.load(f)

In [11]:
results = {}
get_relevant_sentences(cs)
calculate_recall(cs, results)

0.3456723891273248

In [12]:
results = {}
get_relevant_sentences(cst)
calculate_recall(cst, results)

0.4351230425055927